# 02. Faster Facial Landmark Detection

## 어떻게 facial landmark detector의 속도를 향상시킬 수 있을까?

앞에서 배운 대로 facial landmark detection을 사용하면 다소 속도가 느려서 real time으로 사용하기 조금 버거울 수 있습니다. 그러면 어떻게 개선을 해야 할까요? 

Facial Landmark Detection을 하기 위해서는 2가지 step을 가져야 합니다.
1. Face Detection
2. Landmark Detection

과연 이 둘중에서 어떤 것이 좀 더 연산하는 데 시간이 걸릴까요?
사실 Facial Landmark Detection 이지만 Landmark Detection을 하는 데에는 1ms 정도 밖에 걸리지 않습니다. 즉, Face Detector의 성능에 따라 전체 성능이 좌우 됩니다.

## 1. Resize Frame

얼굴 검출 속도는 이미지의 해상도에 따라 달라지며, 해상도가 더 낮은 이미지에서는 더 작은 범위의 얼굴을 찾습니다. 저 해상도로 변경 시 단점은 더 작은 얼굴을 놓치지만 많은 응용 프로그램에서 카메라/웹캠을 보는 사람은 소수의 사람 또는 한명이 대부분이기 때문에 사실상 효과적입니다. <br>
즉, 얼굴 인식 속도를 높이는 쉬운 방법은 프레임 크기를 조정하는 것입니다. 내 웹캠은 720p (즉, 1280 × 720) 해상도로 비디오를 녹화하고 고정된 높이로 이미지의 크기를 조정하고 이에 따라 너비를 변경합니다. 얼굴 검출 이후 얻은 bounding box는 좌표를 원래 프레임 크기 조정에 사용된 scale로 나눠서 다시 크기를 조정해야합니다. 이를 통해 사이즈를 원복 시킬 수 있습니다.

## 2. Skip Frame

일반적으로 웹캠은 30fps로 비디오를 녹화합니다. 하지만 일반적인 응용 프로그램에서는 웹캠 바로 앞에 앉아 있고 사람 또는 사물이 많이 움직이지 않습니다. 따라서 모든 프레임에서 얼굴을 감지 할 필요가 없습니다. 예를 들어 3 프레임마다 Face Detection을 수행하면 거의 3배 가까이 가속화 할 수 있습니다

In [1]:
import cv2, dlib
import sys
from renderFace import renderFace

1. 이제 VideoCapture 객체를 만들고 첫 번째 프레임을 읽습니다.

2. 프레임 높이의 크기를 조정할 크기를 RESIZE_HEIGHT로 계산합니다. 100 프레임을 처리하는 데 걸리는 시간을 측정합니다. 처리중인 프레임 수를 계산합니다. 코드 블록의 마지막 두 줄에서 프레임 카운터와 틱 카운터를 초기화합니다.우리는 함수 실행 시간을 계산하기 위해 getTickCount와 getTickFrequency를 사용합니다.

3. 정지 신호가 나올 때까지 카메라 피드에서 프레임을 읽습니다. 프레임의 높이가 고정된 높이로 조정되고 이미 캡처한 첫 번째 프레임을 사용하여 RESIZE_SCALE을 계산하였습니다.

4. 크기가 조정 된 이미지에서 얼굴 검출기를 실행하면 이미지에있는 모든 얼굴에 직사각형이 표시됩니다.그런 다음 각 얼굴을 하나씩 선택하고 이미지와 이 얼굴에 해당하는 사각형을 사용하여 facial landmark detection을 실행합니다. 얼굴 인식을 위해 크기가 조정된 이미지를 사용 했으므로 RESIZE_SCALE을 사용하여 얼굴 사각형을 확대합니다. 마지막으로 얼굴에 얼굴 윤곽을 그리고 이미지에 현재 fps를 표시해 보겠습니다.

5. 마지막으로 OpenCV 함수인 getTickCount와 getTickFrequency를 사용하여 함수 실행 시간을 계산할 수 있습니다.

In [2]:
try:
    # 1.
    # Create an imshow window 
    winName = "Fast Facial Landmark Detector"
    PREDICTOR_PATH = "pre_trained/68_face_landmarks.dat"    
    
    # Create a VideoCapture object
    cap = cv2.VideoCapture(0)
    
    # Check if OpenCV is able to read feed from camera
    if cap.isOpened() is False :
        print("Unable to connect to camera")
        sys.exit()
    
    fps = 30.0
    # Get first frame
    ret, im = cap.read()
    
    RESIZE_HEIGHT = im.shape[0] / 2
    SKIP_FRAMES = 3
    
    # 2.
    # We will use a fixed height image as input to face detector
    if ret == True:
        height = im.shape[0]
        # calculate resize scale
        RESIZE_SCALE = float(height) / RESIZE_HEIGHT
        size = im.shape[0:2]
    else:
        print("Unable to read frame")
        sys.exit()
        
    # Load face detection and pose estimation models
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(PREDICTOR_PATH)
    # initiate the tickCounter
    t = cv2.getTickCount()
    count = 0
    
    # 3.
    # Grab and process frames until the main window is closed by the user.
    while True:
        if count == 0:
            t = cv2.getTickCount()
        # Grab a frame
        ret, im = cap.read()
        # create imSmall by resizing image by resize scale
        imSmall = cv2.resize(im, None, 
                             fx = 1.0 / RESIZE_SCALE, 
                             fy = 1.0/RESIZE_SCALE, interpolation = cv2.INTER_LINEAR)
        
        # Process frames at an interval of SKIP_FRAMES.
        # This value should be set depending on your system H/W & camera fps.
        # To reduce computations, this value should be increased
        if count % SKIP_FRAMES == 0 :
            # Detect faces
            faces = detector(imSmall, 0)
        
        # Iterate over faces
        for face in faces:
            # Since we ran face detection on a resized image,
            # WE will scale up coordinates of face rectangle
            newRect = dlib.rectangle(int(face.left() * RESIZE_SCALE),
                                     int(face.top() * RESIZE_SCALE),
                                     int(face.right() * RESIZE_SCALE),
                                     int(face.bottom() * RESIZE_SCALE))
            
            # Find face landmarks by providing rectangle for each face
            shape = predictor(im, newRect)
            renderFace(im, shape)
            
        # Put fps at which we are processing camera feed on frame
        cv2.putText(im, "{0:.2f}-fps".format(fps), (50, size[0] - 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 3)
        # Display it all on the screen
        cv2.imshow(winName, im)
        
        # wait for keypress
        key = cv2.waitKey(1) & 0xFF
        
        # Stop the program
        if key == 27 : #ESC
            sys.exit()
            
        # 5.
        # increment frame counter
        count = count + 1
        # calculate fps at an interval of 100 frames
        if count == 100:
            t = (cv2.getTickCount() - t) / cv2.getTickFrequency()
            fps = 100.0 / t
            count = 0
        
    cv2.destroyAllWindows()
    cap.release()
    
except Exception as e:
    print(e)

SystemExit: 

C:\Users\infoe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
